# Table of contents
* [Create dataframe from scratch](#from_scratch)
* [Common dataframe functionalities](#common_functionalities)
    * [Very Common dataframe functionalities](#very_common_functionalities)
    * [Duplicated Data](#duplicated_data)
    * [First stats and info](#first_stats)
    * [Adding Columns/Raws](#adding)
* [Indexing](#indexing)
* [Advanced Selecting](#advanced_selecting)
    * [Conditionning](#conditionning)
    * [Filtering](#filtering)
    * [Sorting](#sorting)
* [Renaming indexes and columns](#renaming)
* [Grouping the data](#grouping)
* [Handling missing data](#missing_data)
* [Combining Data](#combining_data)
* [Date Formatting](#date_formatting)
* [Reshaping data](#reshaping)
* [Apply Functions](#apply_functions)
* [Working with text data](#text_data)
* [Seaborn](#seaborn)
* [Workflow in Titanic example](#titanic)
    * [Data Analysis](#data_analysis)
    * [Banding Features](#banding_features)
    * [Features Importance: permutation](#feature_importance)
    * [Partial Dependance Plot](#partial_plot)

In [ ]:
import pandas as pd
import numpy as np # necessity as pandas is built on np
import matplotlib.pyplot as plt
import seaborn as sns

# Create dataframe from scratch <a class="anchor" id="from_scratch"></a>

In [ ]:
data = {'Country': ['Belgium', 'India', 'Brazil'],
 'Capital': ['Brussels', 'New Delhi', 'Brasília'],
 'Population': [11190846, 1303171035, 207847528]}
df_sample = pd.DataFrame(data)
df_sample

# Common dataframe functionalities <a class="anchor" id="common_functionalities"></a>

## Very Common functionalities <a class="anchor" id="very_common_functionalities"></a>

In [ ]:
df = pd.read_csv('../input/train.csv') # read csv file

In [ ]:
df.shape

In [ ]:
df.head() # see top 5 rows of data

In [ ]:
df.dtypes # see datatype of each variable

In [ ]:
df.columns # column names

## Duplicated data <a class="anchor" id="duplicated_data"></a>

In [ ]:
df.Sex.unique()
df.nunique() # unique value for each variable
(df.nunique())["Sex"]
sum(df.PassengerId.duplicated()) # check if there is a duplicate: should be 0 if not.
df.loc[:,['Sex', 'Embarked']].drop_duplicates()

## First simple stats and info <a class="anchor" id="first_stats"></a>

In [ ]:
df.info() # not null part is very useful to see how many nulls are there in data

In [ ]:
df.describe()
df.Sex.value_counts()
df.Fare.mean()

## Adding columns/raws <a class="anchor" id="adding"></a>

In [ ]:
df.assign(FareSquared = df["Fare"]**2)[:2]
new_df = df.copy()
new_df.Sex = new_df.Sex.fillna("male")
new_df = new_df.assign(Male = (new_df.Sex == "male"))
new_df.Male = new_df.Male.astype(int)
# append for raws

## Manipulations

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

embarked_encoded = pd.DataFrame(data = ['Apple', 'Orange', 'Banana'], columns = ["names"])
embarked_encoded["cat"] = label_encoder.fit_transform(embarked_encoded.names)
embarked_encoded

# Indexing <a class="anchor" id="indexing"></a>

## iloc: integer location

In [ ]:
df.iloc[1:4, 2:6] # indexes are maintained. Can reset_index() to start index from 0  

## loc : label name of columns, index

In [ ]:
df.loc[1:2,'Name':"Age"] # here row indexes are numbers but column indexes are name of columns 

# Advanced selecting <a class="anchor" id="advanced_selecting"></a>

## Conditionning <a class="anchor" id="conditionning"></a>

In [ ]:
# select rows with either sex as female or Pclass as 1
df[(df.Sex == 'female') | (df.iloc[:,2] == 1) ].iloc[:3] # () are important

In [ ]:
# first 3 rows of gives all columns which have all string values or all int > 1 values
df.loc[:,(df > 1).all()][:3]
# first 3 rows of all columns which have all not null values
df.loc[:,(df.notnull().all() )][:3]
# first 3 rows of all columns which have atleast 1 null value
df.loc[:, df.isnull().any()][:3]

In [ ]:
# fraction of males with Age > 25, df.shape[0] -> number of rows 
sum((df.Age > 25) & (df.Sex == 'male'))/df.shape[0]
# number of people who survived and were not in class 3
sum((df.Survived != 0) & (~(df.Pclass == 3)) )

In [ ]:
#isin
df[df.Pclass.isin([0,1])].head()

## Filtering <a class="anchor" id="filtering"></a>

In [ ]:
# filter all rows which have Age > Passenger ID
df.query('Age > PassengerId')

# filter only sex and age columns (first 2 rows)
df.filter(items=['Age', 'Sex'])[:2]

# filter only 0 and 5 row index
df.filter(items=[0,5], axis=0)

# first 2 rows of column names ending with "ed" (think of past tense)
df.filter(like = 'ed', axis=1)[:2]

## Sorting <a class="anchor" id="sorting"></a>

In [ ]:
df.sort_values(by=['Pclass', 'Age'], ascending=False,na_position='first')

# Renaming indexes and columns <a class="anchor" id="renaming"></a>

## Change the index

In [ ]:
# setting the index as the ticket column
df.set_index('Ticket')[:2]

In [ ]:
# can set multiple columns as index also. Just pass them in list
# Setting Ticket and Name as index
df.set_index(['Ticket', 'Name'])[:2]

## Reset index: can reset index back to 0....nrows-1

In [ ]:
df_index = df.set_index(['Ticket', 'Name'])

In [ ]:
df_index.reset_index()[:2]

## Rename columns

In [ ]:
df.rename(columns={'Name': 'Whats_name', 'Fare':'Price'})[:2]

In [ ]:
# can use some mapper function also. default axis='index' (0)
df.rename(mapper=str.lower, axis='columns')[:2]

# Grouping the data <a class="anchor" id="grouping"></a>

In [ ]:
df.groupby(by = ['Sex', 'Survived']).mean() #.loc[:,'Age']

In [ ]:
# can group by indexes also by using levels= 
# useful when we have multindexes
# can use agg function with lambda func

df_index = df.set_index(['Sex', 'Pclass'])
df_index.groupby(level=[0,1]).agg({'Fare': lambda x: sum(x)/len(x), # this is also just mean actually
                                  'Age' : np.mean})

In [ ]:
df_index.groupby(level=[0,1]).transform(lambda x: sum(x)/len(x)).head()

In [ ]:
df.groupby(['Pclass', 'Sex']).apply(lambda df: df.loc[df.Fare.idxmax()])

In [ ]:
df.groupby(['Pclass', 'Sex']).agg([len, min, max]).Fare

# Handling missing data <a class="anchor" id="missing_data"></a>

## Detect missing values

In [ ]:
df[pd.isnull(df.Age)]

## Drop raws with na

In [ ]:
# how=any -> row with any column = NA
df.dropna(axis=0,  how='any').shape
# how=any -> row with all columns = NA
df.dropna(axis=0, how='all').shape

## Drop columns with na

In [ ]:
# drops column which have any row of NA
[set(df.columns) - set(df.dropna(axis=1, how='any').columns)]

## Fill missing values

In [ ]:
# replace with mean of that column
# can put any specific value also
# would not work for columns with string type like Cabin

df.fillna(np.mean)[:1]
df.Age.fillna("Unknown")

# Combining Data <a class="anchor" id="combining_data"></a>

## Example dataframe


In [ ]:
data1 = pd.DataFrame({'x1': list('abc'), 'x2': [11.432, 1.303, 99.906]})

In [ ]:
data2 = pd.DataFrame({'x1': list('abd'), 'x3': [20.784,  np.NaN, 20.784]})

In [ ]:
data1

In [ ]:
data2

## Merge

In [ ]:
merged_inner = pd.merge(left=data1, right=data2, left_on='x1', right_on='x1')
merged_inner

In [ ]:
merged_left = pd.merge(left=data1, right=data2, how='left', left_on='x1', right_on='x1')
merged_left

In [ ]:
#with index as key
inner_concat = pd.concat([data1, data2], axis=1, join='inner')
inner_concat

In [ ]:
# inner join when both table have that key (like sql)

data1.merge(data2, how='inner', on='x1')

In [ ]:
# outer joins on all keys in both df and creates NA

data1.merge(data2, how='outer', on='x1')

can also use `join` but `merge` is faster. just use merge

In [ ]:
# if columns overlap, have to specify suffix as it makes for all

data1.join(data2, on='x1', how='left', lsuffix='L')

## Concatenate

In [ ]:
# Stack the DataFrames on top each other
vertical_stack = pd.concat([data1, data2], axis=0)
vertical_stack

In [ ]:
# Place the DataFrames side by side
horizontal_stack = pd.concat([data1, data2], axis=1)
horizontal_stack

Notice that it has index duplicates as it maintain original df index

Can use `ignore_index=True` to make index start from 0

In [ ]:
pd.concat([data1, data2], axis=0, ignore_index=True)

In [ ]:
data2.loc[3] = ['g', 500] # adding new row
data2

In [ ]:
# join over axis=1, i.e columns combine 

pd.concat([data1, data2], axis=1)

# Date formatting <a class="anchor" id="date_formatting"></a>

* `to_datetime()` -> convert whatever format argument to datetime (obviously that can be parsed to datetime)
* `date_range()` -> generates datetime data
* `Datetimeindex` -> datetypeindex data

In [ ]:
pd.to_datetime('2018-2-19')

In [ ]:
# gives datetimeindex format

pd.date_range('2018-4-18', periods=6, freq='d')

In [ ]:
data1['date'] = pd.date_range('2018-4-18', periods=3, freq='d')

In [ ]:
data1

In [ ]:
data1.date

In [ ]:
pd.DatetimeIndex(data1.date)

# Reshaping data <a class="anchor" id="reshaping"></a>

In [ ]:
# index = new index, columns = new_columns, values = values to put

df.pivot(index='Sex', columns = 'PassengerId', values = 'Age')

In [ ]:
# Stack: convert whole df into 1 long format
df.stack()

# Iteration <a class="anchor" id="iterating"></a>
To get column/row indexes, series pair. 

* `iteritems()` for column-index, series
* `iterrows()` for row-index, series

In [ ]:
list(df.Sex.iteritems())[:5]

In [ ]:
list(df.iterrows())[0]

# Apply functions <a class="anchor" id="apply_functions"></a>

* `apply` -> apply function over df
* `apply_map` -> apply function elementwise (for each series of df. think of column wise)

In [ ]:
# function squares when type(x) = float, cubes when type(x) = int, return same when other

f = lambda x: x**2 if type(x) == float else x**3 if type(x) == int else x


In [ ]:
# whole series is passed

df.Fare.apply(f)[:3]
Fare_mean = df.Fare.mean()
def remean_Fare(row):
    row.Fare = row.Fare - Fare_mean
    return row
(df.apply(remean_Fare, axis='columns')).Fare.mean()

In [ ]:
# elements are passed

df.applymap(f)[:3]

# Working with text data <a class="anchor" id="text_data"></a>
What all can we do when we have string datatype in pandas dataframe/series ?

#### `str` 

Working with string format in pandas series/df 

We can do:
* `str.upper()/lower()` to convert string into upper or lower case
* `str.len()` to find the length of sting
* `str.strip()/lstrip()/rstrip()` to strip spaces
* `str.replace()` to replace anything from string
* `str.split()` to split words of string or using some other delimiter
* `str.get()` to access elements in slit list 
* `str.resplit()` spit in reverse order of string based on some delimiter
* `str.extract()` extract specific thing from string. alphabet or number

Let's see how to use all that in pandas series. Keep in mind pandas DataFrame has no attribute called `str` and works on Series object only. So, grab column of df, then apply `str`


In [ ]:
# converts all rows into lower

df.Name.str.lower().head()

In [ ]:
# converts all rows into upper 

df.Sex.str.upper().head()

In [ ]:
# counts all the characters including spaces

df.Name.str.len().head()

In [ ]:
# splits strings in each row over whitespaces ()
# expand=True : expand columns
# pat = regex to split on

df.Name.str.split(pat=',',expand=True).head().rename(columns={0:'First_Name', 1: 'Last_Name'})

In [ ]:
# splits strings in each row over whitespaces ()
# expand=False : doesn't expand columns
# pat = regex to split on

df.Name.str.split(expand=False).head()

In [ ]:
# replace Mr. with empty space

df.Name.str.replace('Mr.', '').head()

In [ ]:
# get() is used to get particular row of split

df.Name.str.split().get(1)

In [ ]:
df.Name[:10]

In [ ]:
# Extract just last name

df.Name.str.extract('(?P<Last_Name>[a-zA-Z]+)', expand=True).head()

# Seaborn <a class="anchor" id="seaborn"></a>
[cheat sheet](https://storage.googleapis.com/kaggle-forum-message-attachments/713144/14742/seaborn_sheat_cheet.pdf)

# Workflow: Titanic example <a class="anchor" id="titanic"></a>

see https://www.kaggle.com/startupsci/titanic-data-science-solutions

## Data Analysis <a class="anchor" id="data_analysis"></a>

In [ ]:
df[["Sex", "Survived", "Fare"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
g = sns.FacetGrid(df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
grid = sns.FacetGrid(df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
grid = sns.FacetGrid(df, col='Pclass', hue='Survived')
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

## Banding features <a class="anchor" id="banding_features"></a>

In [ ]:
df['AgeBand'] = pd.cut(df['Age'], 5)
df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
df.loc[ df['Age'] <= 16, 'Age'] = 0
df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
df.loc[ df['Age'] > 64, 'Age']
df.head()

## Feature importance: permutation <a class="anchor" id="feature_importance"></a>

https://www.kaggle.com/dansbecker/permutation-importance

## Partial dependance plot <a class="anchor" id="partial_plot"></a>

https://www.kaggle.com/dansbecker/partial-plots